In [17]:
import os.path
import time
import gmail
import base64 
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://mail.google.com/"]
creds = get_authentication()

def get_authentication():

  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "./storage/credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return creds


In [4]:
def search_messages(creds, search_string):
    
    try:
        service = build("gmail", "v1", credentials = creds)
        results = service.users().messages().list(userId ='me', q=search_string).execute()
        try:
            messages = results.get('messages', [])
            print(messages)
        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""
    except (errors.HttpError, error):
        print("An error occured: %s") % error



In [5]:
def get_profile(creds):
    try:
        service = build("gmail", "v1", credentials = creds)
        results = service.users().getProfile(userId = 'me').execute()
        
        print("Email-Id :")
        print(results["emailaddress"])
    
    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

In [6]:
def get_labels(creds):    
    try:
        # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        results = service.users().labels().list(userId="me").execute()
        labels = results.get("labels", [])

        print("Labels:")
        for label in labels:
            print(label["name"])

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


In [7]:
def get_mssgIds(creds):   
    
    try:
        ids=[] 
        service = build("gmail", "v1", credentials=creds)
        results = service.users().messages().list(userId="me", maxResults=5).execute()
        try:
            messages = results.get('messages', [])
        
            for message in messages:
                ids.append(message["id"])
        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")
    return ids

In [8]:
def get_messages(creds):
    try:
        
        service = build("gmail", "v1", credentials=creds)
        results = service.users().messages().get(userId="me", id= "18d131a209077d9").execute()
        try:
            print(results["snippets"])
        except KeyError:
            print("WARNING: the search queried returned 0 results")
            print("returning an empty string")
            return ""

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


In [75]:
def check_mail(creds):

    global msg

    try:
        #Call the Gmail API
        service = build('gmail', 'v1', credentials = creds)

        results = service.users().messages().list(userId='me', labelIds=['INBOX'], q="is:unread", maxResults = 5 ).execute()
        messages = results.get('messages', [])

        if not messages:
            print("you have no new messages")
        else:
            message_count = 0
            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()
                
                message_count = message_count + 1
                #change to return statement for GUI 
            print("You have " + str(message_count) + " unread messages.")

            new_message_choice = input("Would youb like to see your unread messages:").lower()
            #Comment all below for GUI
            if new_message_choice == "yes" or "y":
                for message in messages:
                    msg = service.users().messages().get(userId='me', id=message['id']).execute()
                    email_data = msg['payload']['headers']
                    for values in email_data:
                        name = values["name"]
                        if name == "From":
                            from_name = values["value"]
                            print("You have a new message from: "+ from_name)
                            print(msg['snippet'] + "...")
                            print("\n")
                            time.sleep(1)

            else:
                print("Good-Bye_____:)")

    
    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

 

In [3]:
def check_drafts(creds):

    global dft

    try:
        #Call the Gmail API
        service = build('gmail', 'v1', credentials = creds)

        results = service.users().drafts().list(userId='me' ).execute()
        drafts = results.get('drafts', [])

        draft_count = 0
        for draft in drafts:
            dft = service.users().drafts().get(userId='me', id=draft['id']).execute()
            print(dft)
            draft_count = draft_count + 1
            #change to return statement for GUI 
        print("You have " + str(draft_count) + " drafts.")

    
    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

 

In [ ]:
def delete_drafts(creds):
     
    try:
        #Call the Gmail API
        service = build('gmail', 'v1', credentials = creds)
        
        results = service.users().drafts().delete(userId = 'me', id = ).execute()

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

In [ ]:
def send_mail(creds):
    

In [4]:
def create_draft(creds):
 

    try:
        #Call the Gmail API
        service = build('gmail', 'v1', credentials = creds)
        mimeMessage = MIMEMultipart()
        mimeMessage['from'] = '11tworldaayush@gmail.com'
        mimeMessage['subject'] ='this is not good'
        mimeMessage['to'] = 'aayush28819@gmail.com'

        msg_body = 'Email Body Testing'
        mimeMessage.attach(MIMEText(msg_body,'plain'))
        raw_string = base64.urlsafe_b64encode(mimeMessage.as_bytes()).decode()
        
        results = service.users().drafts().create(userId = 'me', body = {'message': {'raw':raw_string }}).execute()

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")




In [16]:
check_drafts(creds)

{'id': 'r-5660023371828761737', 'message': {'id': '18d35a649e6c5c64', 'threadId': '18d35a649e6c5c64', 'labelIds': ['DRAFT'], 'snippet': 'Email Body Testing', 'payload': {'partId': '', 'mimeType': 'multipart/mixed', 'filename': '', 'headers': [{'name': 'Received', 'value': 'from 1043565051188 named unknown by gmailapi.google.com with HTTPREST; Tue, 23 Jan 2024 01:28:26 -0800'}, {'name': 'Content-Type', 'value': 'multipart/mixed; boundary="===============2312620842099724369=="'}, {'name': 'MIME-Version', 'value': '1.0'}, {'name': 'from', 'value': '11tworldaayush@gmail.com'}, {'name': 'subject', 'value': 'this is not good'}, {'name': 'to', 'value': 'aayush28819@gmail.com'}, {'name': 'Date', 'value': 'Tue, 23 Jan 2024 01:28:26 -0800'}, {'name': 'Message-Id', 'value': '<CAAvtsUUPrH2z1+frXkbTYpXSrN9ENDG_+VuTcpTmn438gxr_-w@mail.gmail.com>'}], 'body': {'size': 0}, 'parts': [{'partId': '0', 'mimeType': 'text/plain', 'filename': '', 'headers': [{'name': 'Content-Type', 'value': 'text/plain; char